In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
from pathlib import Path
from copy import deepcopy
from functools import partial

import numpy as np
import pandas as pd

from bellem.utils import set_seed, jprint
from bellem.musique.singlehop import benchmark
from bellem.musique.qa import answer_question_cte

set_seed(89)

In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
def perfect_retrieval(docs, query):
    return [doc for doc in docs if doc['is_supporting']]

In [5]:
N_RUNS = 3

In [6]:
from bellem.musique.constants import ABLATION_RECORD_IDS

df = pd.read_json('../../data/generated/musique-common/base-dataset-validation.jsonl', orient='records', lines=True)
df = df.set_index('id', drop=False).loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)
# df = df.sample(10)

print(df.shape)
df.head()

(100, 8)


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Mahmoud Mirza >> ...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar]
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'KKVU >> licensed ...",Berrien County,[Berrien County],True,[Berrien County]
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'Pa Sak Jolasid Da...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River]
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Tebesa Nemine >> ...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith]
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Which performer r...",Snapper Foster,[Snapper Foster],True,[Snapper Foster]


In [7]:
with open("./cte-few-shot-examples.json") as f:
    few_shot_examples = json.load(f)


len(few_shot_examples)

37

In [8]:
selected_ids = [
    "2hop__272543_126102",
    "2hop__54719_159941",
    "2hop__16777_419765",
    "2hop__834974_332063",
    "2hop__823584_776926",
    "2hop__784447_126070",
]
selected_few_shot_examples = [example for example in few_shot_examples if example["id"] in selected_ids]

In [9]:
from itertools import combinations

example_combinations = []
for i in range(1, 3):
    example_combinations.extend(list(combinations(selected_few_shot_examples, i)))

len(example_combinations)

21

In [10]:
results = []

for examples in tqdm(example_combinations):
    example_ids = tuple([example["id"] for example in examples])
    qa_func = partial(answer_question_cte, examples=examples, completion_kwargs={'temperature': 0.1})
    for i in range(1, N_RUNS+1):
        df_cte, scores = benchmark(df, qa_func, perfect_retrieval, ignore_errors=False)
        results.append({**scores, 'examples': example_ids, "run": i})

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
report_df = pd.DataFrame.from_records(results, columns=['examples', 'run', 'exact_match', 'f1'])
agg_report_df = report_df.drop(columns=['run']).groupby(['examples']).agg(['min', 'mean', 'max', 'std'])

In [13]:
from datetime import datetime
suffix = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
report_df.to_json(f'./cte-prompt-optim-results-{suffix}.jsonl', orient='records', lines=True)

In [14]:
agg_report_df.sort_values(('f1', 'mean'), ascending=False)

exact_match                  \
                                                   min      mean   max   
examples                                                                 
(2hop__16777_419765, 2hop__834974_332063)         0.61  0.623333  0.64   
(2hop__16777_419765, 2hop__823584_776926)         0.60  0.623333  0.65   
(2hop__272543_126102, 2hop__54719_159941)         0.54  0.583333  0.61   
(2hop__834974_332063,)                            0.59  0.606667  0.62   
(2hop__272543_126102,)                            0.59  0.610000  0.64   
(2hop__272543_126102, 2hop__834974_332063)        0.58  0.593333  0.61   
(2hop__16777_419765, 2hop__54719_159941)          0.57  0.586667  0.60   
(2hop__823584_776926, 2hop__834974_332063)        0.58  0.580000  0.58   
(2hop__16777_419765,)                             0.61  0.623333  0.64   
(2hop__834974_332063, 2hop__784447_126070)        0.60  0.603333  0.61   
(2hop__16777_419765, 2hop__784447_126070)         0.55  0.556667  0.56   
(2hop__272543_126102, 2hop__823584_776926)        0.55  0.556667  0.57   
(2hop__54719_159941,)                             0.56  0.570000  0.58   
(2hop__823584_776926,)                            0.56  0.570000  0.58   
(2hop__16777_419765, 2hop__272543_126102)         0.58  0.590000  0.60   
(2hop__54719_159941, 2hop__784447_126070)         0.54  0.563333  0.58   
(2hop__54719_159941, 2hop__834974_332063)         0.56  0.566667  0.57   
(2hop__823584_776926, 2hop__784447_126070)        0.55  0.560000  0.57   
(2hop__823584_776926, 2hop__54719_159941)         0.54  0.546667  0.55   
(2hop__272543_126102, 2hop__784447_126070)        0.55  0.563333  0.58   
(2hop__784447_126070,)                            0.53  0.540000  0.55   

                                                            f1            \
                                                 std       min      mean   
examples                                                                   
(2hop__16777_419765, 2hop__834974_332063)   0.015275  0.724454  0.738216   
(2hop__16777_419765, 2hop__823584_776926)   0.025166  0.706255  0.733355   
(2hop__272543_126102, 2hop__54719_159941)   0.037859  0.689240  0.714325   
(2hop__834974_332063,)                      0.015275  0.701629  0.712523   
(2hop__272543_126102,)                      0.026458  0.690160  0.711160   
(2hop__272543_126102, 2hop__834974_332063)  0.015275  0.698946  0.707568   
(2hop__16777_419765, 2hop__54719_159941)    0.015275  0.694939  0.706125   
(2hop__823584_776926, 2hop__834974_332063)  0.000000  0.701984  0.705284   
(2hop__16777_419765,)                       0.015275  0.691264  0.704250   
(2hop__834974_332063, 2hop__784447_126070)  0.005774  0.687376  0.696176   
(2hop__16777_419765, 2hop__784447_126070)   0.005774  0.681636  0.690939   
(2hop__272543_126102, 2hop__823584_776926)  0.011547  0.683859  0.690922   
(2hop__54719_159941,)                       0.010000  0.685730  0.690605   
(2hop__823584_776926,)                      0.010000  0.686033  0.687743   
(2hop__16777_419765, 2hop__272543_126102)   0.010000  0.681692  0.686897   
(2hop__54719_159941, 2hop__784447_126070)   0.020817  0.672828  0.683832   
(2hop__54719_159941, 2hop__834974_332063)   0.005774  0.671603  0.679631   
(2hop__823584_776926, 2hop__784447_126070)  0.010000  0.668429  0.677418   
(2hop__823584_776926, 2hop__54719_159941)   0.005774  0.673763  0.677182   
(2hop__272543_126102, 2hop__784447_126070)  0.015275  0.651017  0.672625   
(2hop__784447_126070,)                      0.010000  0.651788  0.655203   

                                                                
                                                 max       std  
examples                                                        
(2hop__16777_419765, 2hop__834974_332063)   0.763407  0.021847  
(2hop__16777_419765, 2hop__823584_776926)   0.756225  0.025252  
(2hop__272543_126102, 2hop__54719_159941)   0.730192  0.021977  
(2hop__834974_332063,)                      0.72359

In [15]:
agg_report_df.sort_values(('exact_match', 'mean'), ascending=False)

exact_match                  \
                                                   min      mean   max   
examples                                                                 
(2hop__16777_419765,)                             0.61  0.623333  0.64   
(2hop__16777_419765, 2hop__834974_332063)         0.61  0.623333  0.64   
(2hop__16777_419765, 2hop__823584_776926)         0.60  0.623333  0.65   
(2hop__272543_126102,)                            0.59  0.610000  0.64   
(2hop__834974_332063,)                            0.59  0.606667  0.62   
(2hop__834974_332063, 2hop__784447_126070)        0.60  0.603333  0.61   
(2hop__272543_126102, 2hop__834974_332063)        0.58  0.593333  0.61   
(2hop__16777_419765, 2hop__272543_126102)         0.58  0.590000  0.60   
(2hop__16777_419765, 2hop__54719_159941)          0.57  0.586667  0.60   
(2hop__272543_126102, 2hop__54719_159941)         0.54  0.583333  0.61   
(2hop__823584_776926, 2hop__834974_332063)        0.58  0.580000  0.58   
(2hop__54719_159941,)                             0.56  0.570000  0.58   
(2hop__823584_776926,)                            0.56  0.570000  0.58   
(2hop__54719_159941, 2hop__834974_332063)         0.56  0.566667  0.57   
(2hop__272543_126102, 2hop__784447_126070)        0.55  0.563333  0.58   
(2hop__54719_159941, 2hop__784447_126070)         0.54  0.563333  0.58   
(2hop__823584_776926, 2hop__784447_126070)        0.55  0.560000  0.57   
(2hop__16777_419765, 2hop__784447_126070)         0.55  0.556667  0.56   
(2hop__272543_126102, 2hop__823584_776926)        0.55  0.556667  0.57   
(2hop__823584_776926, 2hop__54719_159941)         0.54  0.546667  0.55   
(2hop__784447_126070,)                            0.53  0.540000  0.55   

                                                            f1            \
                                                 std       min      mean   
examples                                                                   
(2hop__16777_419765,)                       0.015275  0.691264  0.704250   
(2hop__16777_419765, 2hop__834974_332063)   0.015275  0.724454  0.738216   
(2hop__16777_419765, 2hop__823584_776926)   0.025166  0.706255  0.733355   
(2hop__272543_126102,)                      0.026458  0.690160  0.711160   
(2hop__834974_332063,)                      0.015275  0.701629  0.712523   
(2hop__834974_332063, 2hop__784447_126070)  0.005774  0.687376  0.696176   
(2hop__272543_126102, 2hop__834974_332063)  0.015275  0.698946  0.707568   
(2hop__16777_419765, 2hop__272543_126102)   0.010000  0.681692  0.686897   
(2hop__16777_419765, 2hop__54719_159941)    0.015275  0.694939  0.706125   
(2hop__272543_126102, 2hop__54719_159941)   0.037859  0.689240  0.714325   
(2hop__823584_776926, 2hop__834974_332063)  0.000000  0.701984  0.705284   
(2hop__54719_159941,)                       0.010000  0.685730  0.690605   
(2hop__823584_776926,)                      0.010000  0.686033  0.687743   
(2hop__54719_159941, 2hop__834974_332063)   0.005774  0.671603  0.679631   
(2hop__272543_126102, 2hop__784447_126070)  0.015275  0.651017  0.672625   
(2hop__54719_159941, 2hop__784447_126070)   0.020817  0.672828  0.683832   
(2hop__823584_776926, 2hop__784447_126070)  0.010000  0.668429  0.677418   
(2hop__16777_419765, 2hop__784447_126070)   0.005774  0.681636  0.690939   
(2hop__272543_126102, 2hop__823584_776926)  0.011547  0.683859  0.690922   
(2hop__823584_776926, 2hop__54719_159941)   0.005774  0.673763  0.677182   
(2hop__784447_126070,)                      0.010000  0.651788  0.655203   

                                                                
                                                 max       std  
examples                                                        
(2hop__16777_419765,)                       0.724486  0.017758  
(2hop__16777_419765, 2hop__834974_332063)   0.763407  0.021847  
(2hop__16777_419765, 2hop__823584_776926)   0.756225  0.025252  
(2hop__272543_126102,)                      0.73349

In [25]:
from collections import defaultdict

example_score_mapping = defaultdict(list)

for _, row in report_df.iterrows():
    for example in row['examples']:
        example_score_mapping[example].append(row['f1']) 

sorted([(np.array(scores).mean(), id) for id, scores in example_score_mapping.items()], reverse=True)

[(0.7099637152970486, '2hop__16777_419765'),
 (0.7065663965663966, '2hop__834974_332063'),
 (0.6972495837495838, '2hop__272543_126102'),
 (0.6953172629839296, '2hop__823584_776926'),
 (0.6919499204499204, '2hop__54719_159941'),
 (0.6793656281989615, '2hop__784447_126070')]

In [24]:
selected_ids = [
    "2hop__16777_419765",
    "2hop__823584_776926"
]
with open("selected-cte-few-shot-examples.json", "w") as f:
    json.dump([example for example in few_shot_examples if example["id"] in selected_ids], f, indent=2)